In [1]:
import numpy as np
import pandas as pd
from gurobipy import *
np.random.seed(1)

In [2]:
def Corte_Entrada(i,A):
    Ce = []
    for (x,y) in A:
        if y== i:
            Ce = Ce + [(x,y)]
    return Ce

def Corte_Salida(i):
    Cs = []
    for (x,y) in A:
            if x==i:
                Cs = Cs + [(x,y)]
    return Cs

def Corte_Entrada_Nodo(i,A):
    Ce = []
    for (x,y) in A:
        if y== i:
            Ce = Ce + [x]
    return Ce

def Corte_Salida_Nodo(i,A):
    Cs = []
    for (x,y) in A:
            if x==i:
                Cs = Cs + [y]
    return Cs


## MODELO 2 CON ASIGANCIÓN SOLUCIONES

In [3]:
def Trabajo1 (gap=0, time = 3600,Sol =()):
    np.random.seed(1)
    m = Model()
    # Crear las variables
    print("CREANDO VARIABLES")
    x = m.addVars([(k, m, i, j) for (k, m, i, j) in K], name="x", vtype=GRB.BINARY)
    y = m.addVars([(k, m, j) for k in range (1,n1+1) for m in M[k] for j in V_D], name="y", vtype=GRB.BINARY)
    a = m.addVars([(k, m, i, d) for k in range (1,n1+1) for m in M[k] for i in V for d in range(1,1+n2)] , name = "a", vtype = GRB.INTEGER, lb=0) #CAMBIAR D
    u = m.addVars( V, name="u", vtype = GRB.INTEGER, lb=1, ub= n)
    m.update()
    
    #Función Objetivo
    t_V = {(k, m, i, j): c[i,j] for(k,m,i,j) in K}
    C_M ={ (k, m, "O"+str(k), i): c_M[k,m] for k in range(1,n1+1) for m in M[k] for i in V}
    
    print("CREANDO FUNCION OBJ")
    m.setObjective(x.prod(t_V) + x.prod(C_M) , GRB.MINIMIZE)

    #Restricciones
    
    print("CREANDO RESTRICCIONES")
    
    for k,b,i,j in K:
        if type(i) == str or type(j) == str:
            continue
        m.addConstr(
            (u[j]  >= u[i] + (1 + n)*x[k,b,i,j] - n)
        )
    
    m.addConstrs(
        ( x.sum(k, m,"O"+str(k) ,"*")  <= 1 for k in range(1,n1+1) for m in M[k]) 
    )

    m.addConstrs(
        quicksum(x[k, m, j[0], "D"+str(p)]  for p in range(1,n2+1) for j in Corte_Entrada("D"+str(p), At) ) <=1  for k in range(1,n1+1) for m in M[k]
    )
    
    m.addConstrs(
        ( x.sum(k, m,"*" ,i)  == y[k, m, i]  for k in range(1,n1+1) for m in M[k] for i in V) 
    ) 
    
    m.addConstrs(
        ( x.sum(k, m,"*","D"+str(p) )  == y[k, m, "D"+str(p)] for k in range(1,n1+1) for m in M[k] for p in range(1,n2+1)) 
    )
    
    m.addConstrs(
        ( x.sum(k, m, "*",s)  == x.sum(k, m ,s,"*") for k in range(1,n1+1) for m in M[k] for s in V)
    )

#     m.addConstrs(
#         ( x.sum(k, "*","O"+str(k),"*") <= len(M[k]) for k in range(1,n1+1))
#     )
    
    m.addConstrs(
        (a.sum(k, m, "*", p) <= Dem[p]*y[k, m, "D"+str(p)] for k in range(1,n1+1) for m in M[k] for p in range(1,1+n2))
    )
    
    m.addConstrs(
        (a.sum("*", "*", i, "*") <= OF[i] for i in V )
    )


    m.addConstrs(
        (a.sum(k, m,"*", "*") <= Q[k, m]  for k in range(1,n1+1) for m in M[k])
    )

    m.addConstrs(
        (a[k, m, i, p]<= OF[i]*y[k, m, i] for p in range(1,n2+1)  for k in range(1,n1+1) for m in M[k] for i in V )
    )

    m.addConstrs(
        (a.sum("*", "*", "*", p) == Dem[p] for p in range(1,n2+1))
    )

    m.Params.MIPGap = gap
    m.Params.TimeLimit= time
    print("OPTIMIZE")
    
    
    if len(Sol) == 0:
        m.optimize()
    else:

        for (k,m1,i,j) in K:
            x[k,m1,i,j].Start = 0
            
        for k in range (1,n1+1):
            for m1 in M[k]:
                for j in V_D:
                    y[k,m1,j].Start = 0
        
        for k in range (1,n1+1): 
            for m1 in M[k]: 
                for i in V: 
                    for d in range(1,1+n2):
                        a[k,m1,i,d].Start = 0
            
        
        for key in Sol[0].keys():
            O = key.split("-")[0]
            Ve = int(key.split("-")[-1])
            for arista in Sol[0][key]:
                #PARA ARISTAS (X)
                x[int(O[1:]), Ve, arista[0], arista[-1]].Start = 1
                
                if type(arista[0]) != str:
                    y[int(O[1:]), Ve, arista[0]].Start = 1
                    
                if type(arista[-1]) == str :
                    y[int(O[1:]), Ve, arista[-1]].Start = 1

            for producto in Sol[1][key]:
                #Para Productos (a)
                destino = producto[-1][1:]
                a[int(O[1:]), Ve, producto[0] , int(destino) ].Start = producto[1]  #CAMBIAR SI CAMBIA LA CREACION DE LOS a's
                
        m.update()   
        m.optimize()
    
    print("-"*20)

    clasi = {}
    par = {}
    print("GUARDANDO SOLUCIONES")
    if m.status == GRB.Status.TIME_LIMIT or  m.status == GRB.Status.OPTIMAL:
        par = {"Obj":m.objval, "GAP":abs(m.objval-m.ObjBoundC) /abs(m.objval), "Time": m.Runtime}
    # Recuperar los valores de las variables
        vx = m.getAttr('x', x)
        print('Arcos seleccionados:')
        for (k,m1,i,j) in K:
            if vx[k,m1,i,j] >= 0.99:
                print('Deposito {} usa el auto{}:= {} -> {}'.format(k,m1, i, j))
                if ("O"+str(k)+"-"+str(m1) in clasi.keys() )== False:
                    clasi.update({"O"+str(k)+"-"+str(m1) : [(i,j)]})
                else:
                    clasi["O"+str(k)+"-"+str(m1)]  = clasi["O"+str(k)+"-"+str(m1)] + [(i,j)]
                        
                
    print("-"*20)
    
    
    producto = {}
    if m.status == GRB.Status.TIME_LIMIT or  m.status == GRB.Status.OPTIMAL:
         for k in range (1,n1+1):
                for m1 in M[k]:
                    for i in V :
                        for d in range(1,1+n2):
                            if a[k,m1,i,d].x>0:
                                c_prod = a[k,m1,i,d].x
                                if (("O"+str(k),m1) in producto.keys() )== False:
                                        producto.update({("O"+str(k),m1): [(i, c_prod,d)] })
                                else:
                                    producto[("O"+str(k),m1)] = producto[("O"+str(k),m1)] + [(i, c_prod,d)]
                                print("El auto {} del deposito {} recoge {} del producto en {} para {}".format(m1,k,c_prod,i,d))
                        
    if m.status == GRB.Status.TIME_LIMIT or  m.status == GRB.Status.OPTIMAL:
        for k in range (1,n1+1):
            for m1 in M[k]: 
                for j in V_D:
                    if y[k,m1,j].x>0.99:
                        print("el auto {} del deposito {} visita {}".format(m1, k, j))
    
    return clasi, producto, par

## LECTURA DE LAS SOLUCIONES DE LAS HEURÍSTICAS

In [4]:
f = open("SolHeu.txt","r")
stra = f.read()

In [5]:
import ast
SolucionesInstancias = ast.literal_eval(stra)

## CORRER LA RESOLUCIÓN DE LAS INTANCIAS CON SOLUCIÓN ASIGNADA

In [6]:
# import pandas as pd
from IPython.display import Markdown


df_instancias = pd.read_excel("InstanciasFinal.xlsx",sheet_name= "Sheet1", header=0, index_col=[0,1,2])


Instancias = {5: {11: [17], 14: [17], 17: [2], 18: [6]},
 10: {11: [10], 15: [4], 17: [9], 21: [11]},
 15: {6: [5], 7: [6], 18: [7], 22: [9]},
 20: {12: [2], 13: [8], 19: [10], 24: [4]},
 25: {11: [12], 16: [9], 17: [17], 24: [17]},
 30: {10: [2], 15: [3], 20: [12], 24: [16]},
 35: {6: [15], 15: [2], 19: [5], 23: [19]},
 40: {6: [4], 7: [8], 13: [8], 15: [15]},
 45: {14: [13], 19: [9], 23: [14], 25: [15]},
 50: {13: [13], 14: [14], 16: [13], 21: [10]},
 55: {9: [6], 12: [9], 23: [9], 24: [15]},
 60: {10: [6], 17: [20], 18: [2], 23: [18]},
 65: {15: [9], 19: [12], 22: [15], 25: [19]}}
    

for j in df_instancias.columns:
    s = df_instancias[j]
    for m in list(Instancias.keys()):
        for i in list(Instancias[m].keys()):
            for k in Instancias[m][i]:
                s[m,i,k] = list([int(s[m,i,k].split(",")[0][1:]), int(s[m,i,k].split(",")[1][:-1])+1]) #mas 1 por como se generan los #aleatorios


df_primeras = df_instancias

prob = 0.5

DatosInstancias3 = {}
for n in list(Instancias.keys()):
    V = tuplelist(range(1,n+1))
    np.random.seed(1)
    a1 = {(i,j): np.random.randint(35,151) for i in V for j in V if (i!=j and np.random.random()<prob)}

    for i,j in list(a1.keys()):
        if (j,i) in list(a1.keys()) and i<j:
            a1[j,i] = a1[i,j]

    a = {**a1}
    A = list(a.keys())
    for n1 in list(Instancias[n].keys()):
        O = ["O"+str(i) for i in range(1,n1+1)]
        np.random.seed(1)
        Ao = [ (i,j) for i in O for j in V if np.random.random()<0.85 ]
        np.random.seed(1)
        b1 = {(i,j): np.random.randint(35,150) for (i,j) in Ao }
        V_O = [i for i in range(1,n+1)]+O
        
        for n2 in Instancias[n][n1]:
            display (Markdown('<span style="color: #ff0000">{},{} y {}</span> '.format(n,n1,n2)))
            np.random.seed(1)
            #Creación de D
            D = ["D"+str(i) for i in range(1,n2+1)]
            np.random.seed(1)
            Ad = [ (i,j) for i in V for j in D if np.random.random()<0.85 ]
            np.random.seed(1)
            b2 = {(i,j): np.random.randint(35,150)  for (i,j) in Ad }
            V_D = [i for i in range(1,n+1)]+D

            At,c= multidict({**a, **b1, **b2})
            #Vehículos
            np.random.seed(1)
            M = {k: [i for i in range(1,
                                                  np.random.randint(df_primeras.loc[n].loc[n1]["Rango de \nVehículo para cada deposito"][n2][0]+1,
                                                                    df_primeras.loc[n].loc[n1]["Rango de \nVehículo para cada deposito"][n2][1]+1))] for k in range(1,n1+1) }
            np.random.seed(1)
            c_M = { (k, m):  np.random.randint(350,500) for k in range(1,n1+1) for m in M[k]}
            
            
            
            
            #Creación K
            K = []
            for k in range(1,n1+1):
                for m in M[k]:
                    K = K + [(k, m, j[0], j[1]) for j in Ao if "O"+str(k) == j[0]  ]
                    K = K + [(k,m,i,j) for (i,j) in A+Ad]
                    
            #Capacidades de los vehículos
            np.random.seed(1)
            Q = {(k, m) : np.random.randint(df_primeras.loc[n].loc[n1]["Rango de \ncapacidad"][n2][0], df_primeras.loc[n].loc[n1]["Rango de \ncapacidad"][n2][1]) for k in range(1,n1+1) for m in M[k]}

            #Oferta
            np.random.seed(1)
            OF = {i: np.random.randint(df_primeras.loc[n].loc[n1]["Oferta para\n cada lugar de abastecimiento"][n2][0], df_primeras.loc[n].loc[n1]["Oferta para\n cada lugar de abastecimiento"][n2][1]) for i in V}

            #Demanda
            np.random.seed(1)
            Dem = {p :np.random.randint(df_primeras.loc[n].loc[n1]["Rango Demanda"][n2][0], df_primeras.loc[n].loc[n1]["Rango Demanda"][n2][1]) for p in range(1,n2+1) }
            
            
            Solu = SolucionesInstancias[n,n1,n2]
            
            
            print("Modelo2")
            Ruta2, Pro2, Par2 = Trabajo1(Sol= Solu)
            
            DatosInstancias3.update({(n,n1,n2): [Ruta2, Pro2, Par2] })


<span style="color: #ff0000">5,11 y 17</span> 

Modelo2
Set parameter Username
Academic license - for non-commercial use only - expires 2023-04-21
CREANDO VARIABLES
CREANDO FUNCION OBJ
CREANDO RESTRICCIONES
Set parameter MIPGap to value 0
Set parameter TimeLimit to value 3600
OPTIMIZE
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 5350 rows, 7264 columns and 31545 nonzeros
Model fingerprint: 0xe294a956
Variable types: 0 continuous, 7264 integer (4114 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [4e+01, 6e+02]
  Bounds range     [1e+00, 5e+00]
  RHS range        [1e+00, 5e+02]

User MIP start produced solution with objective 18906 (0.02s)
Loaded user MIP start with objective 18906

Presolve removed 8 rows and 8 columns
Presolve time: 0.03s
Presolved: 5342 rows, 7256 columns, 31529 nonzeros
Variable types: 0 continuous, 7256 integer (4106 binary)

Root relaxation: objective 9.981108e+03, 656

    38    46 10021.4081    7  622 18348.0000 10018.5738  45.4%   475   15s
H   99   103                    18335.000000 10018.5738  45.4%   343   17s
   135   152 10043.8865   20  454 18335.0000 10018.5738  45.4%   346   20s
H  206   214                    18332.000000 10018.5738  45.3%   308   21s
H  355   357                    18325.000000 10018.5738  45.3%   264   24s
   356   400 10100.4537   53  158 18325.0000 10018.5738  45.3%   266   25s
H  405   452                    18292.000000 10018.5738  45.2%   247   25s
H  591   638                    18267.000000 10018.5738  45.2%   190   27s
  1241  1372 11032.7201  312   63 18267.0000 10018.5738  45.2%   108   30s
  1988  1994 16092.2940  740   26 18267.0000 10018.6640  45.2%  81.1   35s
  2503  2585 10142.9281   39  359 18267.0000 10018.6640  45.2%  81.0   40s
  2852  2767 10434.0605  197  420 18267.0000 10018.6640  45.2%  78.0   45s
  2869  2778 16082.3959  647  500 18267.0000 10067.1259  44.9%  77.5   50s
  2887  2790 10188.8065  

  3843  2898 12205.6882   37 1396 18252.0000 12205.6882  33.1%  57.9  522s
  3845  2899 12207.2663  248 1463 18252.0000 12207.2663  33.1%  57.8  526s
  3850  2902 12207.8129  118 1536 18252.0000 12207.8129  33.1%  57.7  530s
  3853  2904 12209.9796  261 1421 18252.0000 12209.9796  33.1%  57.7  536s
  3855  2906 12211.5146   10 1464 18252.0000 12211.5146  33.1%  57.7  540s
  3861  2910 12212.2762  286 1545 18252.0000 12212.2762  33.1%  57.6  545s
  3865  2912 15177.7789  592 1374 18252.0000 12215.2664  33.1%  57.5  551s
  3870  2916 12953.6617  404 1387 18252.0000 12217.9103  33.1%  57.4  558s
  3871  2916 12219.0104  379 1355 18252.0000 12219.0104  33.1%  57.4  560s
  3877  2920 12219.8276  469 1408 18252.0000 12219.8276  33.0%  57.3  565s
  3879  2922 15190.1009  584 1204 18252.0000 12227.0177  33.0%  57.3  571s
  3883  2924 12228.9433  149 1219 18252.0000 12228.9433  33.0%  57.3  575s
  3890  2929 12234.9920  287 1194 18252.0000 12234.9920  33.0%  57.2  581s
  3893  2931 15763.6881  

  4515  3347 13440.0754  135 1278 18252.0000 13440.0754  26.4%   201 1077s
  4519  3350 13443.0022   13 1306 18252.0000 13443.0022  26.3%   201 1080s
  4526  3354 13445.6075  346 1267 18252.0000 13445.6075  26.3%   201 1085s
  4533  3359 13449.2203  188 1280 18252.0000 13449.2203  26.3%   200 1090s
  4541  3364 13453.1940  269 1229 18252.0000 13453.1940  26.3%   200 1095s
  4552  3372 13456.8538  197 1212 18252.0000 13456.8538  26.3%   199 1102s
  4556  3374 13459.1309  287 1245 18252.0000 13459.1309  26.3%   199 1105s
  4566  3381 13464.2003   15 1174 18252.0000 13464.2003  26.2%   199 1111s
  4571  3384 13468.2272  379 1258 18252.0000 13468.2272  26.2%   199 1115s
  4581  3391 13472.7804   32 1195 18252.0000 13472.7804  26.2%   198 1121s
  4586  3394 13477.4347   26 1212 18252.0000 13477.4347  26.2%   198 1125s
  4596  3401 13481.2407  460 1145 18252.0000 13481.2407  26.1%   197 1130s
  4606  3408 13485.3535   86 1233 18252.0000 13485.3535  26.1%   197 1135s
  4612  3412 13491.3928  

...

     0     0 10808.3964    0 1350 17088.0000 10808.3964  36.7%     - 2161s
     0     0 10818.3085    0 1521 17088.0000 10818.3085  36.7%     - 2214s
     0     0 10818.3791    0 1506 17088.0000 10818.3791  36.7%     - 2217s
     0     0 10826.2142    0 1433 17088.0000 10826.2142  36.6%     - 2272s
     0     0 10826.4294    0 1379 17088.0000 10826.4294  36.6%     - 2300s
     0     0 10838.3481    0 1603 17088.0000 10838.3481  36.6%     - 2348s
     0     0 10838.4963    0 1624 17088.0000 10838.4963  36.6%     - 2353s
     0     0 10847.4033    0 1438 17088.0000 10847.4033  36.5%     - 2404s
     0     0 10847.5096    0 1472 17088.0000 10847.5096  36.5%     - 2410s
     0     0 10853.0227    0 1396 17088.0000 10853.0227  36.5%     - 2447s
H    0     0                    17012.000000 10853.0227  36.2%     - 2540s
     0     0 10853.0884    0 1370 17012.0000 10853.0884  36.2%     - 2543s
     0     0 10857.6775    0 1552 17012.0000 10857.6775  36.2%     - 2585s
     0     0 10857.8358  

<span style="color: #ff0000">65,25 y 19</span> 

Modelo2
CREANDO VARIABLES
CREANDO FUNCION OBJ
CREANDO RESTRICCIONES
Set parameter MIPGap to value 0
Set parameter TimeLimit to value 3600
OPTIMIZE
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 200358 rows, 259288 columns and 1340095 nonzeros
Model fingerprint: 0x3e7db06e
Variable types: 0 continuous, 259288 integer (187593 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [4e+01, 6e+02]
  Bounds range     [1e+00, 7e+01]
  RHS range        [1e+00, 1e+02]

User MIP start produced solution with objective 20658 (0.23s)
Loaded user MIP start with objective 20658

Presolve time: 2.31s
Presolved: 200358 rows, 259288 columns, 1162246 nonzeros
Variable types: 0 continuous, 259288 integer (187593 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf. 

     0     0 11142.5842    0 3278 20213.0000 11142.5842  44.9%     - 2034s
     0     0 11154.6282    0 3561 20213.0000 11154.6282  44.8%     - 2133s
     0     0 11154.9430    0 3660 20213.0000 11154.9430  44.8%     - 2445s
     0     0 11154.9469    0 3653 20213.0000 11154.9469  44.8%     - 2447s
     0     0 11165.5511    0 3556 20213.0000 11165.5511  44.8%     - 2541s
     0     0 11165.8662    0 3754 20213.0000 11165.8662  44.8%     - 2559s
     0     0 11165.8994    0 3679 20213.0000 11165.8994  44.8%     - 2564s
     0     0 11165.9107    0 3662 20213.0000 11165.9107  44.8%     - 2567s
     0     0 11175.1640    0 3606 20213.0000 11175.1640  44.7%     - 2623s
     0     0 11175.5365    0 3438 20213.0000 11175.5365  44.7%     - 2638s
     0     0 11175.5969    0 3494 20213.0000 11175.5969  44.7%     - 2642s
     0     0 11175.5986    0 3500 20213.0000 11175.5986  44.7%     - 2643s
     0     0 11190.1263    0 3469 20213.0000 11190.1263  44.6%     - 2713s
     0     0 11190.5385  

el auto 4 del deposito 9 visita 10
el auto 4 del deposito 9 visita D6
el auto 1 del deposito 10 visita 6
el auto 1 del deposito 10 visita 24
el auto 1 del deposito 10 visita 62
el auto 1 del deposito 10 visita D2
el auto 1 del deposito 11 visita 7
el auto 1 del deposito 11 visita D2
el auto 1 del deposito 12 visita 7
el auto 1 del deposito 12 visita 8
el auto 1 del deposito 12 visita 13
el auto 1 del deposito 12 visita 42
el auto 1 del deposito 12 visita D6
el auto 2 del deposito 12 visita 12
el auto 2 del deposito 12 visita D10
el auto 1 del deposito 14 visita 28
el auto 1 del deposito 14 visita D5
el auto 2 del deposito 14 visita 2
el auto 2 del deposito 14 visita 65
el auto 2 del deposito 14 visita D8
el auto 2 del deposito 15 visita 45
el auto 2 del deposito 15 visita 57
el auto 2 del deposito 15 visita D11
el auto 1 del deposito 16 visita 38
el auto 1 del deposito 16 visita 40
el auto 1 del deposito 16 visita 53
el auto 1 del deposito 16 visita D4
el auto 3 del deposito 17 visita 

## PRESENTACIÓN DE INDICADORES DE LAS SOLUCIONES

In [8]:
for i in DatosInstancias3.keys():
    print("____"*20)
    print("INSTANCIA {}".format(i))
    print("Para el modelo 2")
    print(DatosInstancias3[i][-1] )

________________________________________________________________________________
INSTANCIA (5, 11, 17)
Para el modelo 2
{'Obj': 18252.0, 'GAP': 0.01953075895437174, 'Time': 3600.115972518921}
________________________________________________________________________________
INSTANCIA (5, 14, 17)
Para el modelo 2
{'Obj': 18053.0, 'GAP': 0.1339243014846331, 'Time': 3600.183277130127}
________________________________________________________________________________
INSTANCIA (5, 17, 2)
Para el modelo 2
{'Obj': 6903.0, 'GAP': 0.0, 'Time': 260.42689514160156}
________________________________________________________________________________
INSTANCIA (5, 18, 6)
Para el modelo 2
{'Obj': 9478.0, 'GAP': 0.026449585464234173, 'Time': 3600.0653400421143}
________________________________________________________________________________
INSTANCIA (10, 11, 10)
Para el modelo 2
{'Obj': 9094.0, 'GAP': 0.0, 'Time': 1097.877191543579}
__________________________________________________________________________

## GUARDAR SOLUCIONES 

In [33]:
# open file for writing
f = open("SOLUCION_MODELO2_CON_ASIGANCION.txt","w")

# write file
f.write( str(DatosInstancias3) )

# close file
f.close()

## GUARDAD INDICADORES

In [30]:
df_instancias = pd.read_excel("MODELO2_INSTANCIAS_FINAL.xlsx",sheet_name= "Sheet1",header=[0,1],index_col=[0,1,2])



a = pd.Series([DatosInstancias3[i][-1]["Obj"] for i in DatosInstancias3.keys()])
a.index = df_instancias.index
df_instancias[("Con una Solución \n Iniciada","OBJ")] = a

a = pd.Series([DatosInstancias3[i][-1]["GAP"] for i in DatosInstancias3.keys()])
a.index = df_instancias.index
df_instancias[("Con una Solución \n Iniciada","GAP")] = a

a = pd.Series([DatosInstancias3[i][-1]["Time"] for i in DatosInstancias3.keys()])
a.index = df_instancias.index
df_instancias[("Con una Solución \n Iniciada","Tiempo")] = a

In [32]:
df_instancias.to_excel("MODELO2_SOLUCIONES_INCIALES_INSTANCIAS_FINAL.xlsx")